## Noise

- Measure noise from continuum images

In [1]:
import numpy as np
import shutil
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
from prettytable import PrettyTable
from astropy.table import Table



fint = fluxInt.flInt()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/'
regDir = rootDir+'regions/'
imDir = rootDir+'inIms/'

fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir


tableName = rootDir+'inputTable.tbl'
tableFileNames= ascii.read(tableName,format='csv')

outTable = rootDir+'noiseFluxInt.tbl'

print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


	+---------+
	 Inputs loaded
	+---------+


We measure the noise of images from 24 frequency intervals, between 84 MHz and 217 GHz.

In [4]:
totReg = regDir+'ToT.reg'
elReg = regDir+'EL.reg'
wlReg = regDir+'WL.reg'

regions = [totReg, elReg, wlReg]

errPerc=  np.array(tableFileNames.columns[2],dtype=float)

fr=[]
rmsArray= np.zeros([len(regions),len(tableFileNames.columns[0])])
noiseArray= np.zeros([len(regions),len(tableFileNames.columns[0])])

cutoff= np.zeros([len(tableFileNames.columns[0])])

for i in xrange(0,15):

    fileName = imDir+tableFileNames.columns[0][i]

    datas,heads = fint.openFile(fileName)
    datas=np.squeeze(datas)
    heads=fint.cleanHead(heads)

    if not 'CDELT2' in heads:
        heads['CDELT2'] = heads['CD2_2']
        heads['CDELT1'] = heads['CD1_1']
    if 'Frequency' in tableFileNames.dtype.names:
        freq = float(tableFileNames.columns[1][i])*1e6
        fr.append(freq)
    elif 'FREQ' in heads:
        freq = float(heads['FREQ'])
        fr.append(freq)
    elif 'CRVAL3' in heads:
        freq = float(heads['CRVAL3'])
        fr.append(freq)
    else:
        print fileName
        
    beamArea = 2*np.pi*float(heads['BMAJ'])*3600./2.35482*float(heads['BMIN'])*3600./2.35482
    pixArea = -float(heads['CDELT2']*3600.)*float(heads['CDELT1']*3600.)
    numPixBeam= beamArea/pixArea   
    
    for j in xrange (0,len(regions)):
        
        if j==0:
            background, rms, pixels = fint.noiseReg(datas,heads,regions[j])
        else:
            backround, rmsEL, pixels = fint.noiseReg(datas,heads,regions[j])
        
        noiseArray[j,i] = rms*np.sqrt(pixels)/numPixBeam
    
    rmsArray[:,i] = rms

for i in xrange(15,len(tableFileNames)):

    fileName = imDir+tableFileNames.columns[0][i]

    datas,heads = fint.openFile(fileName)
    datas=np.squeeze(datas)
    heads=fint.cleanHead(heads)

    if not 'CDELT2' in heads:
        heads['CDELT2'] = heads['CD2_2']
        heads['CDELT1'] = heads['CD1_1']
    if 'Frequency' in tableFileNames.dtype.names:
        freq = float(tableFileNames.columns[1][i])*1e6
        fr.append(freq)
    elif 'FREQ' in heads:
        freq = float(heads['FREQ'])
        fr.append(freq)
    elif 'CRVAL3' in heads:
        freq = float(heads['CRVAL3'])
        fr.append(freq)
    else:
        print fileName
        
    beamArea = 2*np.pi*float(heads['BMAJ'])*3600./2.35482*float(heads['BMIN'])*3600./2.35482
    pixArea = -float(heads['CDELT2']*3600.)*float(heads['CDELT1']*3600.)
    numPixBeam= beamArea/pixArea   
    
    for j in xrange (0,len(regions)):
        
        if j==0:
            background, rms, pixels = fint.noiseMultiReg(datas,heads,regDir+'noiseReg/')
            rms_or = rms/np.sqrt(pixels)
            noiseArray[0,i] = rms/numPixBeam
        else:
            backround, rmsEL, pixels = fint.noiseReg(datas,heads,regions[j])

            noiseArray[j,i] = rms_or*np.sqrt(pixels)/numPixBeam
    
    rmsArray[:,i] = rms_or
    
print '''\t+---------+\n\t Noise measured\n\t+---------+'''
  

	+---------+
	 Noise measured
	+---------+


**Output tables**:

```
~/Projects/MFS/FornaxA/contData/noiseFluxes.tbl'

```


In [3]:
colNames = ['#Filenames','FlCorr','ErrPerc','rmsT [mJy/beam]', 'rmsEL [mJy/beam]', 'rmsWL [mJy/beam]',
            'noiseToT [Jy]', 'noiseEL [Jy]', 'noiseWL [Jy]']

flCorr = np.array(tableFileNames.columns[1],dtype=float)
errPerc=  np.array(tableFileNames.columns[2],dtype=float)

alldata = np.array([tableFileNames.columns[0],flCorr,errPerc,rmsArray[0,:].T,rmsArray[1,:].T,rmsArray[2,:].T,
                    noiseArray[0,:].T,noiseArray[1,:].T,noiseArray[2,:].T])

tt = Table(alldata.T,names=colNames,meta={'name': 'Noise table'})

ascii.write(tt,outTable, overwrite=True)

t = PrettyTable(colNames)
t.add_row(alldata)


print t
print '''\t+---------+\n\t Tables Saved\n\t+---------+'''


+--------------------------------------------------------------------------+-------------------------------------------------------------------------+---------------------------------------------------------------------------+-----------------------------------------------------------------------+-----------------------------------------------------------------------+-----------------------------------------------------------------------+----------------------------------------------------------------------+--------------------------------------------------------------------+------------------------------------------------------------------------+
|                                #Filenames                                |                                  FlCorr                                 |                                  ErrPerc                                  |                            rmsT [mJy/beam]                            |                            rmsEL [mJy/beam]  